In [4]:
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize


In [5]:
#We retrieve data from the Swiss TXT API
base_football_url = "https://test.sport.api.swisstxt.ch/v1/contests/football/"

#define league
liga = "bundesliga/"
#define season
season = "2022"
#define language
lang = "?lang=DE"


In [6]:
#get season
season_info_raw = requests.get(base_football_url + liga+lang).json()


In [7]:
season_info_raw.keys()
data_season = season_info_raw['phases']

In [8]:
# Normalize the JSON data
normalized_data = json_normalize(data_season)

# Convert the normalized data to a Pandas DataFrame
df_season_raw = pd.DataFrame(normalized_data)

# View the DataFrame
df_season_raw.tail()

/var/folders/zy/65zqs5x55kd6l4xmdbksz289zrrqyz/T/ipykernel_96770/2333008283.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  normalized_data = json_normalize(data_season)


,id,key,displayName,state,isActual,showActuals,type,cesimReference.contestId,cesimReference.masterEventId,cesimReference.eventId,sport.id,sport.key,sport.name
29,4686-58-189,4686-58-189,30. Runde,Planned,True,False,Round,4686,58,189,7,football,Fussball
30,4686-58-190,4686-58-190,31. Runde,Planned,False,False,Round,4686,58,190,7,football,Fussball
31,4686-58-191,4686-58-191,32. Runde,Planned,False,False,Round,4686,58,191,7,football,Fussball
32,4686-58-192,4686-58-192,33. Runde,Planned,False,False,Round,4686,58,192,7,football,Fussball
33,4686-58-193,4686-58-193,34. Runde,Planned,False,False,Round,4686,58,193,7,football,Fussball


In [9]:
df_season_raw['season_id'] = df_season_raw['cesimReference.contestId'].astype(str) + "-" + df_season_raw['cesimReference.masterEventId'].astype(str)
season_ids = list(set(df_season_raw['season_id']))
season_id = season_ids[0]

In [10]:
#get all the games for this season
base_url_eventitems = "https://test.sport.api.swisstxt.ch/v1/eventItems?phaseIds="
season_id = season_ids[0]
start_date = "2016-01-01" #we don't care as long as it includes our chosen season
end_date = "2030-01-01"
games_info_raw = requests.get(base_url_eventitems + season_id +"&fromDate="+start_date +"&toDate="+end_date+lang).json()


In [11]:
container = []

for i in range(0,len(games_info_raw)):
    game = {'id': games_info_raw[i]['id'],
            'round_id': games_info_raw[i]['contestInfo']['phases'][1]['id']}
    container.append(game)

phase_info = pd.DataFrame(container)
    

In [12]:
# Normalize the JSON data
normalized_data = json_normalize(games_info_raw)

# Convert the normalized data to a Pandas DataFrame
df_games_raw = pd.DataFrame(normalized_data)


/var/folders/zy/65zqs5x55kd6l4xmdbksz289zrrqyz/T/ipykernel_96770/3859984260.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  normalized_data = json_normalize(games_info_raw)


In [13]:
phase_info

,id,round_id
0,1751082,4686-58-160
1,1751084,4686-58-160
2,1751086,4686-58-160
3,1751087,4686-58-160
4,1751088,4686-58-160
...,...,...
301,1751383,4686-58-193
302,1751384,4686-58-193
303,1751385,4686-58-193
304,1751386,4686-58-193


In [14]:
cols_to_drop = [col for col in df_games_raw.columns if col.endswith('.it') or col.endswith('.fr')]
df_games_raw.drop(cols_to_drop, axis=1, inplace=True)
pd.set_option('display.max_columns', None)

df_games_raw = pd.merge(df_games_raw, phase_info, on='id')


df_games_raw.head()

,id,type,state,eventItemOrder,pushLiveData,dateTimeInfo.fullDateTime,dateTimeInfo.date,dateTimeInfo.time,displayTitle.de,sport.id,sport.key,sport.name.de,competitor1.id,competitor1.infostradaId,competitor1.name.de,competitor1.shortName.de,competitor1.country,competitor1.isNationalTeam,competitor1.results.main,competitor1.results.firstHalf,competitor1.results.secondHalf,competitor2.id,competitor2.infostradaId,competitor2.name.de,competitor2.shortName.de,competitor2.country,competitor2.isNationalTeam,competitor2.results.main,competitor2.results.firstHalf,competitor2.results.secondHalf,contestInfo.contestSeason.id,contestInfo.contestSeason.key,contestInfo.contestSeason.name.de,contestInfo.contestSeason.season,contestInfo.phases,scores.firstHalf.competitor1,scores.firstHalf.competitor2,scores.firstHalf.formatted.de,scores.secondHalf.competitor1,scores.secondHalf.competitor2,scores.secondHalf.formatted.de,scores.main.competitor1,scores.main.competitor2,scores.main.formatted.de,city.de,stadium.de,spectators.de,referees.de,refereeCountry.de,round_id
0,1751082,Game,Finished,0,True,2022-08-05T20:30:00,Fr 05.08.,20:30,Eintracht Frankfurt - Bayern München,7,football,Fussball,3103,4046,Eintracht Frankfurt,E.Frankfurt,GER,False,1.0,0.0,1.0,3101,4017,Bayern München,B.München,GER,False,6.0,5.0,1.0,4686,bundesliga,Bundesliga,2023,"[{'id': '4686-58', 'key': '4686-58', 'name': {...",0.0,5.0,0 : 5,1.0,1.0,1 : 1,1.0,6.0,1 : 6,Frankfurt am Main,Deutsche Bank Park,51500,Deniz Aytekin,GER,4686-58-160
1,1751084,Game,Finished,0,True,2022-08-06T15:30:00,Sa 06.08.,15:30,Union Berlin - Hertha Berlin,7,football,Fussball,115477,5060,Union Berlin,U.Berlin,GER,False,3.0,1.0,2.0,3102,4257,Hertha Berlin,Hertha BSC,GER,False,1.0,0.0,1.0,4686,bundesliga,Bundesliga,2023,"[{'id': '4686-58', 'key': '4686-58', 'name': {...",1.0,0.0,1 : 0,2.0,1.0,2 : 1,3.0,1.0,3 : 1,Berlin,Stadion An der Alten Försterei,22012,Marco Fritz,GER,4686-58-160
2,1751086,Game,Finished,0,True,2022-08-06T15:30:00,Sa 06.08.,15:30,Mönchengladbach - TSG Hoffenheim,7,football,Fussball,3111,4231,Mönchengladbach,M'gladbach,GER,False,3.0,1.0,2.0,12089,107979,TSG Hoffenheim,Hoffenheim,GER,False,1.0,1.0,0.0,4686,bundesliga,Bundesliga,2023,"[{'id': '4686-58', 'key': '4686-58', 'name': {...",1.0,1.0,1 : 1,2.0,0.0,2 : 0,3.0,1.0,3 : 1,Mönchengladbach,Borussia-Park,49659,Daniel Siebert,GER,4686-58-160
3,1751087,Game,Finished,0,True,2022-08-06T15:30:00,Sa 06.08.,15:30,VfL Wolfsburg - Werder Bremen,7,football,Fussball,3094,100340,VfL Wolfsburg,Wolfsburg,GER,False,2.0,1.0,1.0,3097,4143,Werder Bremen,W.Bremen,GER,False,2.0,2.0,0.0,4686,bundesliga,Bundesliga,2023,"[{'id': '4686-58', 'key': '4686-58', 'name': {...",1.0,2.0,1 : 2,1.0,0.0,1 : 0,2.0,2.0,2 : 2,Wolfsburg,Volkswagen Arena,28015,Sascha Stegemann,GER,4686-58-160
4,1751088,Game,Finished,0,True,2022-08-06T15:30:00,Sa 06.08.,15:30,VfL Bochum - FSV Mainz 05,7,football,Fussball,84760,100020,VfL Bochum,VfL Bochum,GER,False,1.0,1.0,0.0,3120,103110,FSV Mainz 05,Mainz 05,GER,False,2.0,1.0,1.0,4686,bundesliga,Bundesliga,2023,"[{'id': '4686-58', 'key': '4686-58', 'name': {...",1.0,1.0,1 : 1,0.0,1.0,0 : 1,1.0,2.0,1 : 2,Bochum,Vonovia Ruhrstadion,24110,Felix Zwayer,GER,4686-58-160


In [15]:
all_teams = list(set(df_games_raw["competitor1.name.de"]))
all_teams

In [17]:
import pandas as pd

# Create a list of dictionaries for each finished game with the round ID, competitor names, and scores
games = []
for i in range(len(df_games_raw)):
    if df_games_raw.loc[i, 'state'] == 'Finished':
        game = {}
        game['round_id'] = df_games_raw.loc[i, 'round_id']
        game['team1'] = df_games_raw.loc[i, 'competitor1.name.de']
        game['team2'] = df_games_raw.loc[i, 'competitor2.name.de']
        game['team1_score'] = df_games_raw.loc[i, 'competitor1.results.main']
        game['team2_score'] = df_games_raw.loc[i, 'competitor2.results.main']
        games.append(game)

# Create a list of dictionaries for each team with the team name, round ID, points, goals scored, and goals scored against
teams = []
for team in all_teams:
    for i, round_id in enumerate(df_games_raw['round_id'].unique(), start=1):
        team_data = {'team': team, 'round_id': round_id, 'points': 0, 'goals_scored': 0, 'goals_conceded': 0, 'round_num': i}
        teams.append(team_data)

# Calculate the points and other stats for each team in each round based on their scores in the games
for game in games:
    team1_goals = game['team1_score']
    team2_goals = game['team2_score']
    team1_points = 1
    team2_points = 1
    if team1_goals > team2_goals:
        team1_points = 3
        team2_points = 0
    elif team1_goals < team2_goals:
        team1_points = 0
        team2_points = 3
    for team in teams:
        if team['team'] == game['team1'] and team['round_id'] == game['round_id']:
            team['points'] += team1_points
            team['goals_scored'] += team1_goals
            team['goals_conceded'] += team2_goals
        elif team['team'] == game['team2'] and team['round_id'] == game['round_id']:
            team['points'] += team2_points
            team['goals_scored'] += team2_goals
            team['goals_conceded'] += team1_goals

# Create a new DataFrame from the list of team dictionaries
df_stats = pd.DataFrame(teams)


In [18]:
df_stats.sort_values('round_num')

,team,round_id,points,goals_scored,goals_conceded,round_num
0,Werder Bremen,4686-58-160,1,2.0,2.0,1
544,Hertha Berlin,4686-58-160,0,1.0,3.0,1
510,VfB Stuttgart,4686-58-160,1,1.0,1.0,1
102,Union Berlin,4686-58-160,3,3.0,1.0,1
476,1. FC Köln,4686-58-160,3,3.0,1.0,1
...,...,...,...,...,...,...
441,RB Leipzig,4686-58-193,0,0.0,0.0,34
475,VfL Wolfsburg,4686-58-193,0,0.0,0.0,34
509,1. FC Köln,4686-58-193,0,0.0,0.0,34
271,Mönchengladbach,4686-58-193,0,0.0,0.0,34


In [19]:
df_stats# Add up points for each team one round after another
df_stats.sort_values(['team', 'round_num'], inplace=True)
df_stats['total_points'] = df_stats.groupby('team')['points'].cumsum()

# Calculate average points scored for last 5 rounds
df_stats['avg_points_last_5'] = df_stats.groupby('team')['points'].rolling(5, min_periods=1).mean().reset_index(0, drop=True)

# Calculate average points scored for last 10 rounds
df_stats['avg_points_last_10'] = df_stats.groupby('team')['points'].rolling(10, min_periods=1).mean().reset_index(0, drop=True)

# Calculate average goals scored for last 5 rounds
df_stats['avg_goals_scored_last_5'] = df_stats.groupby('team')['goals_scored'].rolling(5, min_periods=1).mean().reset_index(0, drop=True)

# Calculate average goals scored for last 10 rounds
df_stats['avg_goals_scored_last_10'] = df_stats.groupby('team')['goals_scored'].rolling(10, min_periods=1).mean().reset_index(0, drop=True)

# Calculate average goals conceded for last 5 rounds
df_stats['avg_goals_conceded_last_5'] = df_stats.groupby('team')['goals_conceded'].rolling(5, min_periods=1).mean().reset_index(0, drop=True)

# Calculate average goals conceded for last 10 rounds
df_stats['avg_goals_conceded_last_10'] = df_stats.groupby('team')['goals_conceded'].rolling(10, min_periods=1).mean().reset_index(0, drop=True)

# Sort by round_num
df_stats.sort_values(['team','round_num'], inplace=True)
df_stats = df_stats.reset_index()


In [20]:
df_season_small = df_season_raw[["id", "displayName"]]
df_season_small.columns = ["round_id", "displayName"]
df_stats = pd.merge(df_stats, df_season_small)


In [21]:
df_stats['num_rounds_left'] = max(df_stats['round_num'])-df_stats['round_num']
df_stats.head()

,index,team,round_id,points,goals_scored,goals_conceded,round_num,total_points,avg_points_last_5,avg_points_last_10,avg_goals_scored_last_5,avg_goals_scored_last_10,avg_goals_conceded_last_5,avg_goals_conceded_last_10,displayName,num_rounds_left
0,476,1. FC Köln,4686-58-160,3,3.0,1.0,1,3,3.0,3.0,3.0,3.0,1.0,1.0,1. Runde,33
1,340,Augsburg,4686-58-160,0,0.0,4.0,1,0,0.0,0.0,0.0,0.0,4.0,4.0,1. Runde,33
2,578,Bayer Leverkusen,4686-58-160,0,0.0,1.0,1,0,0.0,0.0,0.0,0.0,1.0,1.0,1. Runde,33
3,170,Bayern München,4686-58-160,3,6.0,1.0,1,3,3.0,3.0,6.0,6.0,1.0,1.0,1. Runde,33
4,34,Borussia Dortmund,4686-58-160,3,1.0,0.0,1,3,3.0,3.0,1.0,1.0,0.0,0.0,1. Runde,33


In [22]:
proj_points_avg_last_5 = df_stats['total_points'] + (df_stats['num_rounds_left'] * df_stats['avg_points_last_5'])
df_stats['proj_points_avg_last_5'] = proj_points_avg_last_5
proj_points_avg_last_10 = df_stats['total_points'] + (df_stats['num_rounds_left'] * df_stats['avg_points_last_10'])
df_stats['proj_points_avg_last_10'] = proj_points_avg_last_10
df_stats.head()


,index,team,round_id,points,goals_scored,goals_conceded,round_num,total_points,avg_points_last_5,avg_points_last_10,avg_goals_scored_last_5,avg_goals_scored_last_10,avg_goals_conceded_last_5,avg_goals_conceded_last_10,displayName,num_rounds_left,proj_points_avg_last_5,proj_points_avg_last_10
0,476,1. FC Köln,4686-58-160,3,3.0,1.0,1,3,3.0,3.0,3.0,3.0,1.0,1.0,1. Runde,33,102.0,102.0
1,340,Augsburg,4686-58-160,0,0.0,4.0,1,0,0.0,0.0,0.0,0.0,4.0,4.0,1. Runde,33,0.0,0.0
2,578,Bayer Leverkusen,4686-58-160,0,0.0,1.0,1,0,0.0,0.0,0.0,0.0,1.0,1.0,1. Runde,33,0.0,0.0
3,170,Bayern München,4686-58-160,3,6.0,1.0,1,3,3.0,3.0,6.0,6.0,1.0,1.0,1. Runde,33,102.0,102.0
4,34,Borussia Dortmund,4686-58-160,3,1.0,0.0,1,3,3.0,3.0,1.0,1.0,0.0,0.0,1. Runde,33,102.0,102.0


In [24]:
df_stats[df_stats['team']=="Union Berlin"].head(29)

,index,team,round_id,points,goals_scored,goals_conceded,round_num,total_points,avg_points_last_5,avg_points_last_10,avg_goals_scored_last_5,avg_goals_scored_last_10,avg_goals_conceded_last_5,avg_goals_conceded_last_10,displayName,num_rounds_left,proj_points_avg_last_5,proj_points_avg_last_10
13,102,Union Berlin,4686-58-160,3,3.0,1.0,1,3,3.000000,3.000000,3.000000,3.000000,1.000000,1.000000,1. Runde,33,102.000000,102.000000
31,103,Union Berlin,4686-58-161,1,0.0,0.0,2,4,2.000000,2.000000,1.500000,1.500000,0.500000,0.500000,2. Runde,32,68.000000,68.000000
49,104,Union Berlin,4686-58-162,3,2.0,1.0,3,7,2.333333,2.333333,1.666667,1.666667,0.666667,0.666667,3. Runde,31,79.333333,79.333333
67,105,Union Berlin,4686-58-163,3,6.0,1.0,4,10,2.500000,2.500000,2.750000,2.750000,0.750000,0.750000,4. Runde,30,85.000000,85.000000
85,106,Union Berlin,4686-58-164,1,1.0,1.0,5,11,2.200000,2.200000,2.400000,2.400000,0.800000,0.800000,5. Runde,29,74.800000,74.800000
103,107,Union Berlin,4686-58-165,3,1.0,0.0,6,14,2.200000,2.333333,2.000000,2.166667,0.600000,0.666667,6. Runde,28,75.600000,79.333333
121,108,Union Berlin,4686-58-166,3,2.0,0.0,7,17,2.600000,2.428571,2.400000,2.142857,0.600000,0.571429,7. Runde,27,87.200000,82.571429
139,109,Union Berlin,4686-58-167,0,0.0,2.0,8,17,2.000000,2.125000,2.000000,1.875000,0.800000,0.750000,8. Runde,26,69.000000,72.250000
157,110,Union Berlin,4686-58-168,3,1.0,0.0,9,20,2.000000,2.222222,1.000000,1.777778,0.600000,0.666667,9. Runde,25,70.000000,75.555556
175,111,Union Berlin,4686-58-169,3,2.0,0.0,10,23,2.400000,2.300000,1.200000,1.800000,0.400000,0.600000,10. Runde,24,80.600000,78.200000


In [25]:
df_stats.head()

,index,team,round_id,points,goals_scored,goals_conceded,round_num,total_points,avg_points_last_5,avg_points_last_10,avg_goals_scored_last_5,avg_goals_scored_last_10,avg_goals_conceded_last_5,avg_goals_conceded_last_10,displayName,num_rounds_left,proj_points_avg_last_5,proj_points_avg_last_10
0,476,1. FC Köln,4686-58-160,3,3.0,1.0,1,3,3.0,3.0,3.0,3.0,1.0,1.0,1. Runde,33,102.0,102.0
1,340,Augsburg,4686-58-160,0,0.0,4.0,1,0,0.0,0.0,0.0,0.0,4.0,4.0,1. Runde,33,0.0,0.0
2,578,Bayer Leverkusen,4686-58-160,0,0.0,1.0,1,0,0.0,0.0,0.0,0.0,1.0,1.0,1. Runde,33,0.0,0.0
3,170,Bayern München,4686-58-160,3,6.0,1.0,1,3,3.0,3.0,6.0,6.0,1.0,1.0,1. Runde,33,102.0,102.0
4,34,Borussia Dortmund,4686-58-160,3,1.0,0.0,1,3,3.0,3.0,1.0,1.0,0.0,0.0,1. Runde,33,102.0,102.0


In [26]:
# Calculate goal difference for each game
df_stats['goal_diff_game'] = df_stats['goals_scored'] - df_stats['goals_conceded']

# Group by round and sum goal differences
df_stats['goal_diff'] = df_stats.groupby('team')['goal_diff_game'].cumsum()


df_stats.head()

,index,team,round_id,points,goals_scored,goals_conceded,round_num,total_points,avg_points_last_5,avg_points_last_10,avg_goals_scored_last_5,avg_goals_scored_last_10,avg_goals_conceded_last_5,avg_goals_conceded_last_10,displayName,num_rounds_left,proj_points_avg_last_5,proj_points_avg_last_10,goal_diff_game,goal_diff
0,476,1. FC Köln,4686-58-160,3,3.0,1.0,1,3,3.0,3.0,3.0,3.0,1.0,1.0,1. Runde,33,102.0,102.0,2.0,2.0
1,340,Augsburg,4686-58-160,0,0.0,4.0,1,0,0.0,0.0,0.0,0.0,4.0,4.0,1. Runde,33,0.0,0.0,-4.0,-4.0
2,578,Bayer Leverkusen,4686-58-160,0,0.0,1.0,1,0,0.0,0.0,0.0,0.0,1.0,1.0,1. Runde,33,0.0,0.0,-1.0,-1.0
3,170,Bayern München,4686-58-160,3,6.0,1.0,1,3,3.0,3.0,6.0,6.0,1.0,1.0,1. Runde,33,102.0,102.0,5.0,5.0
4,34,Borussia Dortmund,4686-58-160,3,1.0,0.0,1,3,3.0,3.0,1.0,1.0,0.0,0.0,1. Runde,33,102.0,102.0,1.0,1.0


In [28]:
df_stats['rank_proj5'] = df_stats.sort_values(by=['proj_points_avg_last_5', 'goal_diff'],
               ascending=[False, False]).groupby('round_num').cumcount() + 1
df_stats['rank_proj10'] = df_stats.sort_values(by=['proj_points_avg_last_10', 'goal_diff'],
               ascending=[False, False]).groupby('round_num').cumcount() + 1
df_stats
df_stats

,index,team,round_id,points,goals_scored,goals_conceded,round_num,total_points,avg_points_last_5,avg_points_last_10,avg_goals_scored_last_5,avg_goals_scored_last_10,avg_goals_conceded_last_5,avg_goals_conceded_last_10,displayName,num_rounds_left,proj_points_avg_last_5,proj_points_avg_last_10,goal_diff_game,goal_diff,rank_proj5,rank_proj10
0,476,1. FC Köln,4686-58-160,3,3.0,1.0,1,3,3.0,3.0,3.0,3.0,1.0,1.0,1. Runde,33,102.0,102.0,2.0,2.0,3,3
1,340,Augsburg,4686-58-160,0,0.0,4.0,1,0,0.0,0.0,0.0,0.0,4.0,4.0,1. Runde,33,0.0,0.0,-4.0,-4.0,17,17
2,578,Bayer Leverkusen,4686-58-160,0,0.0,1.0,1,0,0.0,0.0,0.0,0.0,1.0,1.0,1. Runde,33,0.0,0.0,-1.0,-1.0,12,12
3,170,Bayern München,4686-58-160,3,6.0,1.0,1,3,3.0,3.0,6.0,6.0,1.0,1.0,1. Runde,33,102.0,102.0,5.0,5.0,1,1
4,34,Borussia Dortmund,4686-58-160,3,1.0,0.0,1,3,3.0,3.0,1.0,1.0,0.0,0.0,1. Runde,33,102.0,102.0,1.0,1.0,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,135,Union Berlin,4686-58-193,0,0.0,0.0,34,55,0.0,1.0,0.0,0.8,0.0,0.3,34. Runde,0,55.0,55.0,0.0,13.0,3,3
608,543,VfB Stuttgart,4686-58-193,0,0.0,0.0,34,25,0.0,0.5,0.0,0.7,0.0,1.0,34. Runde,0,25.0,25.0,0.0,-15.0,16,16
609,237,VfL Bochum,4686-58-193,0,0.0,0.0,34,27,0.0,0.5,0.0,0.6,0.0,1.0,34. Runde,0,27.0,27.0,0.0,-34.0,15,15
610,475,VfL Wolfsburg,4686-58-193,0,0.0,0.0,34,43,0.0,0.8,0.0,0.8,0.0,0.5,34. Runde,0,43.0,43.0,0.0,14.0,8,8


In [29]:
df_stats[df_stats['team']=="FC Zürich"].head(29)

,index,team,round_id,points,goals_scored,goals_conceded,round_num,total_points,avg_points_last_5,avg_points_last_10,avg_goals_scored_last_5,avg_goals_scored_last_10,avg_goals_conceded_last_5,avg_goals_conceded_last_10,displayName,num_rounds_left,proj_points_avg_last_5,proj_points_avg_last_10,goal_diff_game,goal_diff,rank_proj5,rank_proj10


In [33]:
df_stats[['rank_proj5','team','proj_points_avg_last_5']][df_stats['displayName']=="29. Runde"].sort_values('rank_proj5')

,rank_proj5,team,proj_points_avg_last_5
508,1,Borussia Dortmund,70.0
507,2,Bayern München,66.0
517,3,Union Berlin,65.0
514,4,SC Freiburg,61.0
506,5,Bayer Leverkusen,60.0
513,6,RB Leipzig,57.0
510,7,FSV Mainz 05,54.0
520,8,VfL Wolfsburg,51.0
509,9,Eintracht Frankfurt,44.0
504,10,1. FC Köln,43.0


In [ ]:
df_stats = df_stats[['team', 'round_id', 'points', 'goals_scored', 'goals_conceded',
       'round_num', 'total_points', 'avg_points_last_5', 'avg_points_last_10',
       'avg_goals_scored_last_5', 'avg_goals_scored_last_10',
       'avg_goals_conceded_last_5', 'avg_goals_conceded_last_10',
       'displayName', 'num_rounds_left', 'proj_points_avg_last_5',
       'proj_points_avg_last_10', 'goal_diff_game', 'goal_diff', 'rank_proj5',
       'rank_proj10']]

In [ ]:
df_stats[df_stats['team']=="FC Zürich"].to_csv('fcz.csv',index=False)